# Atomistic Hamiltonians via interfacing with external electronic structure codes

## Table of Content: <a name="TOC"></a>

1. [Defining atomistic Hamiltonians via interfacing Libra with electronic structure packages](#define_atomistic)

 1.1. [Interfacing Libra and PSI4](#interface-psi4)
 
 1.2. [Interfacing Libra and DFTB+](#interface-dftbp)
 
   
2. [PES plotting and scans](#pes)

 2.1. [PES for LiH molecule via EOM-CCSD](#pes_scan-1)
 
 2.2. [PES for HFCO molecule via TD-DFTB](#pes_scan-2)
  

## A. Learning objectives

- to be able to define Hamiltonian computation function based on interface with external electronic structure codes
- to be able to define Hamiltonian computation function based on interface with Psi4 electronic structure code
- to be able to define Hamiltonian computation function based on interface with DFTB+ electronic structure code
- to be able to use the defined atomistic Hamiltonians in computing PES
- to be able to build PES via mapping 1D reaction coordinate onto many-DOF molecular geometry transformation


## B. Use cases

- [Define Libra/Psi4 intraface Hamiltonian](#interface-psi4)
- [Define Libra/DFTB+ intraface Hamiltonian](#interface-dftbp)
- [Plotting LiH EOM-CCSD/sto-3G PES](#pes_scan-1)
- [Plotting HFCO TD-DFTB mapped 1D PES](#pes_scan-2)
- [Generate XYZ trajectory from a list of matrices](#make_path_xyz2)
- [Visualize the MD trajectories with py3Dmol](#md_trajectory-vis)


## C. Functions

- `libra_py`
  - `DFTB_methods`
   - [`read_dftb_output`](#read_dftb_output)
   - `run_dftb_adi` [Ex1](#run_dftb_adi-ex1) | [Ex2](#run_dftb_adi-ex2)
   - [`make_dftb_input`](#make_dftb_input)
  - `dynamics_plotting`
   - `plot_pes_properties` [Ex1](#plot_pes_properties-ex1) | [Ex2](#plot_pes_properties-ex2) 
  - `psi4_methods`
   - `run_psi4_adi` [Ex1](#run_psi4_adi-ex1) | [Ex2](#run_psi4_adi-ex2)
  - `scan`
   - `coords2xyz` [Ex1](#coords2xyz-ex1) | [Ex2](#coords2xyz-ex2)
   - [`make_path_xyz2`](#make_path_xyz2)

## D. Classes and class members
    
None  



In [1]:
import sys
import cmath
import math
import os
import h5py
import psi4

import matplotlib.pyplot as plt   # plots
import numpy as np
%matplotlib inline 


if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
import util.libutil as comn

from libra_py import units
from libra_py.data_visualize import colors, clrs_index
import libra_py.dynamics_plotting as dynamics_plotting
import libra_py.scan as scan
import libra_py.psi4_methods as psi4_methods
import libra_py.DFTB_methods as DFTB_methods

import py3Dmol   # molecular visualization

/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<float, std::all

<a name="define_atomistic"></a>[Back to TOC](#TOC)
## 1. Defining atomistic Hamiltonians via Interfacing Libra with electronic structure packages

In this and the following sections, we are going to adapt the process of defining model Hamiltonians to the realm of atomistic electronic structure calculations.

We are going to preserve the interface of the function that can be passed to the dynamical calculations ( a.k.a. `compute_model`) to what it was in the [model Hamiltonian examples](../1_pes_plotting/tutorial.ipynb), but we will change what is going on inside these `compute_model` functions. 

For refresher on how the function's input and output parameters, please see [this tutorial](../1_pes_plotting/tutorial.ipynb)



<a name="interface-psi4"></a>[Back to TOC](#TOC)
### 1.1. Interfacing Libra with [Psi4](#http://www.psicode.org/)

The function below demonstrates how one can interface Libra with the Psi4 code.

The input parameters are described in the corresponding docstring, here we just discuss some aspects:


1. The user can specify the methods/basis to use for computing the ground and excited state energies and the corresponding gradients. For the full range of options, one is referred to the [Psi4](#http://www.psicode.org/) documentation. Some possible options for computing energies and gradients are:
   
   Ground state energies and gradients:  
    - "pbe/sto-3g"
    - "mp2/aug-cc-pVDZ"
    - "ccsd/aug-cc-pVDZ"
    
   Excited state energies and gradients:  
    - "eom-ccsd/aug-cc-pVDZ"
    
   Excited state energies, no gradients:  
    - "cisd/aug-cc-pVDZ"
    - "adc/aug-cc-pVDZ"
    
2. The `options` parameter is used to further control the execution of the Psi4 calculations, for instance:

   - `'prop_root':istate` - can be used to indicate for which rood (state) we are going to compute properties 
   This is important if we want to obtain state-resolved gradients, for instance
   
   - `'roots_per_irrep':[3, 0, 0, 0]` - one needs to indicate how many roots per each irreducible representation
   of the point group symmetries to which the molecular system belongs we need to compute
   
   - `'reference':'rohf'` - is used to indicate what type of reference wavefunction to use in the excited-states 
   calculations

3. This function defines only a subset of the Hamiltonian properties, so not all types of PES scan or dynamics can be executed with it:

       obj.ham_adi = CMATRIX(nstates, nstates)    
       obj.hvib_adi = CMATRIX(nstates, nstates)            
       obj.d1ham_adi = CMATRIXList();            
    
   The vibronic Hamiltonian, `hvib_adi` in this case contains only the electronic Hamiltonian part and not NACs
 
   More properties can be defined (such as `nac_adi`) in principle. 
 
4. Setting up a temporary file is a conveninet thing - if the files get large (and they can be so easily), you will know where to find it

       psi4.core.set_output_file('tmp.dat',False)
       
5. The geometry of the system for Psi4 is setup with the following block:

       mol = psi4.geometry(F"""
       {charge} {spin_multiplicity}
       {coords_str }

       units bohr
       """)

   To prepare the coordiantes of the xyz file, we use the auxiliary `coords2xyz` function from Libra:
   
       coords_str = scan.coords2xyz(labels, q, indx)
            
6. The energy is computed via:

       E, wfc = psi4.energy(grad_method_gs, molecule=mol, return_wfn=True)
       
   This call also request returning the converged wavefunction (the `wfc` object) - which is needed
   for properties calculations (gradients), but also can be used for NAC calculations


7. The gradients calculations goes like:

       grad = np.asarray(psi4.gradient(grad_method_ex, ref_wfn=wfc))
       
   Note how the previously returned wavefunction is used as the reference one.


8. For convenience, the function defined below is also defined in the `libra_py.psi4_methods.py` module. 

<a name="run_psi4_adi-ex1"></a>

In [2]:
class tmp:
    pass

def run_psi4_adi(q, params_, full_id):
    """
   
    This function executes the Psi4 quantum chemistry calculations and 
    returns the key properties needed for dynamical calculations.

    Args: 
        q ( MATRIX(ndof, ntraj) ): coordinates of the particle [ in Bohr units ]
        params ( dictionary ): model parameters
 
            * **params["labels"]** ( list of strings ): the labels of atomic symbolc - for all atoms,
                and in a order that is consistent with the coordinates (in triples) stored in `q`.
                The number of this labels is `natoms`, such that `ndof` = 3 * `natoms`. [ Required ]
            * **params["nstates"]** ( int ): the total number of electronic states 
                in this model [ default: 1 - just the ground state ]
            * **params["grad_method_gs"]** ( string ):  the name of the methodology to compute the 
                energy and gradients on the ground state [ defaut: "ccsd/sto-3g" ]
                Examples: 
                  "pbe/sto-3g", "mp2/aug-cc-pVDZ", "ccsd/aug-cc-pVDZ" # ground state energies, gradients                   
            * **params["grad_method_ex"]** ( string ):  the name of the methodology to compute the 
                energy and gradients on the excited states [ defaut: "eom-ccsd/sto-3g" ]
                Examples:                                     
                  "eom-ccsd/aug-cc-pVDZ", # excited state energies, gradients                  
                If you need just excited state energies (but not gradients), consider: 
                "cisd/aug-cc-pVDZ", adc/aug-cc-pVDZ                
            * **params["charge"]** ( int ): the total charge of the system [ default: 0 ]
            * **params["spin_multiplicity"]** ( int ): the total spin multiplicity [ default: 1 - singlet ]
            * **params["options"]** ( dictionary ): additional parameters of calculations [ default: empty ]
                Examples: 
                  - {} - noting extra
                  - {'reference':'rohf'}, 
                  - {'roots_per_irrep':[3, 0, 0, 0], 'prop_root':1, 'reference':'rohf'}
                  - {'num_roots':3, 'follow_root':2, 'reference':'rohf'} - for state-resolved gradients
            * **params["verbosity"]** ( int ): the level of output of the execution-related 
                information [ default : 0]
                  
        full_id ( intList ): the "path" to the Hamiltonian in the Hamiltonian's hierarchy. Usually, 
            this is Py2Cpp_int([0, itraj]) - the index of the trajectory in a swarm of trajectories
            
    Returns:       
        PyObject: obj, with the members:

            * obj.ham_adi ( CMATRIX(nstates,nstates) ): adiabatic Hamiltonian
            * obj.hvib_adi ( CMATRIX(nstates,nstates) ): vibronic Hamiltonian in the adiabatic basis
            * obj.d1ham_adi ( list of ndof CMATRIX(nstates, nstates) objects ): 
                derivatives of the adiabatic Hamiltonian w.r.t. the nuclear coordinate            
 
    """

    # Make a copy of the input parameters dictionary
    params = dict(params_)
    
    # Defaults    
    critical_params = [ "labels" ] 
    default_params = { "nstates":1,
                       "grad_method_gs":"ccsd/sto-3g", 
                       "grad_method_ex":"eom-ccsd/sto-3g", 
                       "charge":0, "spin_multiplicity":1,                       
                       "options":{},
                       "verbosity":0
                     }
    comn.check_input(params, default_params, critical_params)
    
    # Extract the key variables
    grad_method_gs = params["grad_method_gs"]
    grad_method_ex = params["grad_method_ex"]
    charge = params["charge"]
    spin_multiplicity = params["spin_multiplicity"]
    labels = params["labels"]    
    nstates = params["nstates"]
    options = params["options"]
    verbosity = params["verbosity"]
    natoms = len(labels)
    ndof = 3 * natoms
    
    
    obj = tmp()
    obj.ham_adi = CMATRIX(nstates, nstates)    
    obj.hvib_adi = CMATRIX(nstates, nstates)            
    obj.d1ham_adi = CMATRIXList();            
    for idof in range(ndof):        
        obj.d1ham_adi.append( CMATRIX(nstates, nstates) )
  

    Id = Cpp2Py(full_id)
    indx = Id[-1]

    # Setup and execute the PSI4 calculations
    psi4.core.set_output_file('tmp.dat',False)    
    coords_str = scan.coords2xyz(labels, q, indx)

    mol = psi4.geometry(F"""
    {charge} {spin_multiplicity}
    {coords_str }

    units bohr
    """)

        
    for istate in range(nstates):

        E, grad = None, None
        if istate == 0:
                        
            # Compute the energy 
            if verbosity >0:
                print("Doing state 0 energy")
                
            E, wfc = psi4.energy(grad_method_gs, molecule=mol, return_wfn=True)

            # Compute force at the converged density
            if verbosity >0:
                print("Doing state 0 gradient")
            grad = np.asarray(psi4.gradient(grad_method_gs, ref_wfn=wfc))            
            
        else:
    
            opt = dict(options)
            opt.update({ 'prop_root':istate, 'roots_per_irrep':[3, 0, 0, 0],  'reference':'rohf'  })
            psi4.set_options(opt)

            # Compute the energy 
            if verbosity >0:
                print(F"Doing state {istate} energy")
                
            E, wfc = psi4.energy(grad_method_ex, molecule=mol, return_wfn=True)

            # Compute force at the converged density
            if verbosity >0:
                print(F"Doing state {istate} gradient")
            
            grad = np.asarray(psi4.gradient(grad_method_ex, ref_wfn=wfc))

        
        obj.ham_adi.set(istate, istate, E * (1.0+0.0j) )
        obj.hvib_adi.set(istate, istate, E * (1.0+0.0j) )        
        for iatom in range(natoms):        
            obj.d1ham_adi[3 * iatom + 0].set(istate, istate, grad[iatom, 0] * (1.0+0.0j) )
            obj.d1ham_adi[3 * iatom + 1].set(istate, istate, grad[iatom, 1] * (1.0+0.0j) )
            obj.d1ham_adi[3 * iatom + 2].set(istate, istate, grad[iatom, 2] * (1.0+0.0j) )
        
                        
    return obj


Below, we are going to demonstrate the use of this function for computing properties of the LiH molecule.

Lets first look what the `coords2xyz` function does:
<a name="coords2xyz-ex1"></a>

In [3]:
%%time 

labels = ["Li", "H"]
q = MATRIX(6,1)
q.set(0,0, 0.0);  q.set(1,0, 0.0); q.set(2,0, 0.0);
q.set(3,0, 0.0);  q.set(4,0, 0.0); q.set(5,0, 1.5);

print( scan.coords2xyz( labels, q, 0) )

Li  0.0 0.0 0.0
H  0.0 0.0 1.5

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.17 ms


Now we can specify some system-specific parameters:

* `labels: ["Li", "H"]` - this is all we have in the LiH molecule

* `nstates: 3` - we are going to compute 3 states (all singlets)

* `verbosity:1` - add some progress printing

<a name="run_psi4_adi-ex2"></a>

In [4]:
%%time 

params_ = { "labels": labels, "nstates":3, "verbosity":1    }
id = Py2Cpp_int([0, 0])
obj_psi4 = run_psi4_adi(q, params_, id )

Doing state 0 energy
Doing state 0 gradient
Doing state 1 energy
Doing state 1 gradient
Doing state 2 energy
Doing state 2 gradient
CPU times: user 2.22 s, sys: 14.2 s, total: 16.5 s
Wall time: 43 s


Repeat the same calculation, but using the "standard" (part of the Libra distribution) code, imported with the `psi4_methods` module:

In [5]:
%%time 

params_ = { "labels": labels, "nstates":3, "verbosity":1    }
id = Py2Cpp_int([0, 0])
obj_psi4 = psi4_methods.run_psi4_adi(q, params_, id )

NameError: name 'comn' is not defined

The returned object contains all the properties defined in the function discussed above.

So, we can now access them (as the class data members) and perhaps store them (using the methods of the CMATRIX class - the data type of the `obj.ham_adi`, etc.) 

In [6]:
prefix = "lih"
obj_psi4.ham_adi.show_matrix(F"{prefix}-ham_adi.txt")
obj_psi4.d1ham_adi[0].show_matrix(F"{prefix}-d1ham_adi-0.txt")
obj_psi4.d1ham_adi[1].show_matrix(F"{prefix}-d1ham_adi-1.txt")

<a name="interface-dftbp"></a></a>[Back to TOC](#TOC)
### 1.2. Interfacing Libra with [DFTB+](#https://www.dftbplus.org/)

In this section, we'll go over the way to create an interface bettween Libra and DFTB+ code. 

The assumption is that we are using DFTB+ in to compute the TD-DFTB excited states via Casida's method. 

For such calculations, we need a template input file `dftb_input_template.hsd`, which is located in this directory and is a part of this tutorial. You can specify most of the details on how to conduct the calculations in this file, except for several places that merely serve as the placeholdels and would be modified in actual calculations:

1. Specification of the input geometry: 

       Geometry = GenFormat {
          <<< "input_name,gen"
       }
   
   
2. Specification of the state of interest:

       StateOfInterest    = some_state_index_here
   
   
3. Specification of the type of the forces to be computed:

       ExcitedStateForces = yes


During the actual execution, the template file will be used to create the actual working input file (always called `dftb_in.hsd`) and these fields would be changed accordingly. 

In addition, depending on your system, you may need to obtain the corresponding Slater-Koster files. In this tutorial, we are looking at the fluoro-formaldehyde HFCO, so we need to use the [halogen-organics SK files set](https://www.dftb.org/parameters/download/halorg/halorg-0-1-cc/)


Before we define the Libra-DFTB+ Hamiltonian computation function, it is convenient to define the axusiliary functions for creating input and reading the output (the `detailed.out`) files associated with DFTB+

As mentioned above, the `make_dftb_input` function simply updates the placeholder fields:

<a name="make_dftb_input"></a>

In [7]:
def make_dftb_input(dftb_input_template_filename, istate):
    """
    This function creates an input file for DFTB+ package from a template file,
    it changes several placeholder lines to ensure the calculations are done
    for required electronic states
    
    Args: 
        dftb_input_template_filename ( strings ): the name of the generic input file (template) 
            for DFTB+ calculations
            
        istate ( int ): the index of the state, for which the calculations will be carried out
        
    Returns:
        None :  just creates the files
        
    """
    
    # Read in the template file
    f = open(dftb_input_template_filename, 'r')
    dftb_input_template = f.readlines()
    f.close() 

    
    nlines = len(dftb_input_template)
    
    # Create the actual output file - just replace few parameters in the template file
    dftb_input = open("dftb_in.hsd","w");     
    
    for i in range( nlines ):                
        
        dftb_input_template_line = dftb_input_template[i].split()
        
        
        if len(dftb_input_template_line) > 0:
                    
            if dftb_input_template_line[0] == "<<<":
                dftb_input.write( '    <<< "tmp.gen"\n ')

            elif dftb_input_template_line[0] == "StateOfInterest":
                dftb_input.write( F"    StateOfInterest    = {istate}\n" )

            elif istate == 0 and dftb_input_template_line[0] == "ExcitedStateForces":
                dftb_input.write( "    ExcitedStateForces = no\n" )

            else:
                dftb_input.write(dftb_input_template[i])
                
        else:
            dftb_input.write(dftb_input_template[i])

                    
    dftb_input.close()


The `read_dftb_output` function reads in the total energy (essentially the reference, ground state energy), the excitation energies, and the corresponding forces on all atoms. 

<a name="read_dftb_output"></a>

In [8]:
def read_dftb_output(natoms, istate):
    """
    This file reads in the total energy (essentially the reference, ground state energy), 
    the excitation energies, and the corresponding forces on all atoms and return them in 
    a digital format for further processing.
    
    Args:
    
        natoms ( int ): the number of atoms in the systemm
        istate ( int ): the index of the calculation - just for archiving purposes
        
    Returns: 
        double, MATRIX(ndof, 1): the total energy of a given electronic state,         
            and the corresponding gradients
    """
    
    # Check the successful completion of the calculations like this:
    if os.path.isfile('detailed.out'):
        #print("Found detailed.out")
        os.system(F"cp detailed.out detailed_{istate}.out")
    else:
        print("\nCannot find the file detailed.out")
        print(F"Hint: Current working directory is: {os.getcwd()}")
        print("Is this where you expect the file detailed.out to be found?")
        print("Exiting program...\n")
        sys.exit(0)
            
    
    ndof = 3 * natoms
    grad = MATRIX(ndof, 1)
    
    
    f = open("detailed.out")
    output = f.readlines()
    f.close()
    
    E = 0.0
    nlines = len(output)
            
    for i in range( nlines ):

        output_line = output[i].split()

        if len( output_line ) >= 2:
            
            if output_line[0] == "Total" and output_line[1] == "Forces":

                for j in range( natoms ):

                    next_lines = output[i+j+1].split()  

                    grad.set( 3*j+0, 0, -float(next_lines[1]) )
                    grad.set( 3*j+1, 0, -float(next_lines[2]) )
                    grad.set( 3*j+2, 0, -float(next_lines[3]) )

            if output_line[0] == "Excitation" and output_line[1] == "Energy:" :
            
                E += float(output_line[2])  # energy in a.u.
                #print(output_line[2])
                
            if output_line[0] == "Total" and output_line[1] == "energy:" :
                
                E += float(output_line[2])  # energy in a.u.
                #print(output_line[2])
    
    #print(F"Final energy = {E}")
    return E, grad          

Finally, we are redy to define the required interface function:

Once the parameters are read in (and verified, default values are used where needed), the flow is rather straightforward:

<a name="coords2xyz-ex2"></a>

1. Create a temporary .xyz file using the `scan.coords2xyz` function:

       coords_str = scan.coords2xyz(labels, q, indx, 1.0/units.Angst)
       f = open("tmp.xyz", "w"); 
       f.write( F"{natoms}\nTemporary xyz file\n{coords_str}" )
       f.close()

   Note the needed units conversion - the DFTB+ expects the .xyz files to list the coordinates in Ansgstrom (at least with the current template), but Libra internally uses Bohrs, so it is likely that the input will be in Bohrs.
   
2. Convert the .xyz file to the .gen file using the **xyz2gen** tool supplied with DFTB+

       os.system(F"{xyz2gen_exe} tmp.xyz")
       
   Here, note the following:
   
   - the intermediate files should be called tmp.xyz and tmp.gen - this is what is expected the generated input file will also list
   
   - the xyz2gen tool should be working properly, so make sure you set up your `xyz2gen_exe` parameter correctly
   
   - also, for the correct execution of the xyz2gen tool, you may need to tweak the supplied code a bit: 
     if you are using conda-installed Python, the default python listed in the tool's shebang line may not be
     the one that you need to use. Then edit the topmost line of the "xyz2gen" file in your DFTB+ installation 
     to change it to point to the correct python executable. For instance, on my laptop, i set it up to something
     like this:
     
         #! /home/alexey/miniconda2/envs/py37/bin/python)

3. Then for each state: 

   - create input files:

         make_dftb_input(dftb_input_template_filename, istate)
         
   - execute the DFTB+ code:
   
         os.system( dftb_exe )
         
     This is where you need the `dftbp_exe` parameter - make sure it points to the correct executable (ending with dftb+)
     
   - read the outputs to extract the needed information:
   
         E, grad = read_dftb_output(natoms, istate)
     
   - store the variables in the allocated memory of the created object


As before, for convenience, the function defined below and the two auxiliary functions defined above are also defined in the `libra_py.DFTB_methods.py` module. 

<a name="run_dftb_adi-ex1"></a>

In [9]:
def run_dftb_adi(q, params_, full_id):
    """
   
    This function executes the DFTB+ quantum chemistry calculations and 
    returns the key properties needed for dynamical calculations.

    Args: 
        q ( MATRIX(ndof, ntraj) ): coordinates of the particle [ units: Bohr ]
        params ( dictionary ): model parameters

            * **params["labels"]** ( list of strings ): the labels of atomic symbolc - for all atoms,
                and in a order that is consistent with the coordinates (in triples) stored in `q`.
                The number of this labels is `natoms`, such that `ndof` = 3 * `natoms`. [ Required ]                
            * **params["nstates"]** ( int ): the total number of electronic states 
                in this model [ default: 1 - just the ground state]                     
            * **params["dftb_input_template_filename"]** ( string ):  the name of the input file template
                [ default: "dftb_input_template.hsd" ]                
            * **params["dftbp_exe"]** ( string ):  the full path to the DFTB+ executable 
                [ defaut: "dftb+" ]
            * **params["xyz2gen_exe"]** ( string ):  the full path to the xyz2gen executable 
                (part of the DFTB+ package) [ defaut: "xyz2gen" ]                
                Note: sometimes, especially if you are using conda-installed Python, you may need to
                edit the "xyz2gen" file in your DFTB+ installation to change the topmost line to 
                point to the correct python executable (e.g. #! /home/alexey/miniconda2/envs/py37/bin/python)            
                            
    Returns:       
        PyObject: obj, with the members:

            * obj.ham_adi ( CMATRIX(nstates,nstates) ): adiabatic Hamiltonian             
            * obj.d1ham_adi ( list of ndof CMATRIX(nstates, nstates) objects ): 
                derivatives of the adiabatic Hamiltonian w.r.t. the nuclear coordinate            
 
    """

    params = dict(params_)
    
    critical_params = [ "labels" ] 
    default_params = { "dftb_input_template_filename":"dftb_input_template.hsd",     
                       "nstates":1,
                       "dftb_exe":"dftb+",  "xyz2gen_exe":"xyz2gen"
                     }
    comn.check_input(params, default_params, critical_params)
        
    labels = params["labels"]      
    dftb_input_template_filename = params["dftb_input_template_filename"]
    nstates = params["nstates"]    
    dftb_exe = params["dftb_exe"]
    xyz2gen_exe = params["xyz2gen_exe"]
    
    natoms = len(labels)
    ndof = 3 * natoms
    
    
    obj = tmp()
    obj.ham_adi = CMATRIX(nstates, nstates)    
    obj.nac_adi = CMATRIX(nstates, nstates)    
    obj.hvib_adi = CMATRIX(nstates, nstates)            
    obj.d1ham_adi = CMATRIXList();            
    for idof in range(ndof):        
        obj.d1ham_adi.append( CMATRIX(nstates, nstates) )
  

    Id = Cpp2Py(full_id)
    indx = Id[-1]

    
    # Make an xyz file
    # since the DFTB+ expects the coordinates in Angstrom, but Libra 
    # goes with atomic units (Bohrs), hence expecting the `q` variable be 
    # in Bohrs, we need to convert the coordinates from Bohrs to Angstroms
    coords_str = scan.coords2xyz(labels, q, indx, 1.0/units.Angst)
    f = open("tmp.xyz", "w"); 
    f.write( F"{natoms}\nTemporary xyz file\n{coords_str}" )
    f.close()
        
    # Convert xyz to gen format: tmp.xyz -> tmp.gen
    # The temporary working file MUST be called "tmp.gen" since this is
    # what the DFTB+ input will list - see the `make_dftbp_input`
    os.system(F"{xyz2gen_exe} tmp.xyz")

        
    for istate in range(nstates):
        
        # Update the input file
        make_dftb_input(dftb_input_template_filename, istate)
        
        # We have written the dftb+ input file for a certain state in nstates. Now we must compute the 
        # state energies and forces. 
        os.system( dftb_exe )
        
        # At this point, we should have the "detailed.out" file created, so lets read it        
        E, grad = read_dftb_output(natoms, istate)

        # Now, populate the allocated matrices                
        obj.ham_adi.set(istate, istate, E * (1.0+0.0j) )
        obj.hvib_adi.set(istate, istate, E * (1.0+0.0j) )        
        for idof in range(ndof):        
            obj.d1ham_adi[idof].set(istate, istate, grad.get(idof, 0) * (1.0+0.0j) )                
    
                    
    return obj


Now, lets give it a try.

In this example, we are going to compute 3 states of the HFCO (fluoroformaldehyde) molecule, so: 

* `"labels":["F", "C", "H", "O"]` - this is the identity and the order of the atoms in our system

* `"nstates":3` - 3 states

Note how we the got the original coordinates in Angstrom, but before the calculations we convert them to Bohrs.

<a name="run_dftb_adi-ex2"></a>

In [10]:
%%time 

params_ = { "labels":["F", "C", "H", "O"], "nstates":3,
            "dftb_exe":"/mnt/c/cygwin/home/Alexey-user/Soft/dftb/dftbplus-19.1.x86_64-linux/bin/dftb+",
            "xyz2gen_exe":"/mnt/c/cygwin/home/Alexey-user/Soft/dftb/dftbplus-19.1.x86_64-linux/bin/xyz2gen"
          }

# This is the input in Angstrom
q = MATRIX(12,1)
q.set(0,0, -0.04746);  q.set(1,0,  0.65920);  q.set(2,0,  1.04080)
q.set(3,0, -0.51816);  q.set(4,0,  0.81540);  q.set(5,0,  0.06880)
q.set(6,0,  0.15264);  q.set(7,0,  1.00050);  q.set(8,0,  -0.77170)
q.set(9,0, -1.71206);  q.set(10,0, 0.78520);  q.set(11,0, -0.06860)
q.show_matrix()

# The input is expected to be in Bohr, so lets convert:
q *= units.Angst

id = Py2Cpp_int([0, 0])
obj_dftb = run_dftb_adi(q, params_, id )

CPU times: user 15.6 ms, sys: 188 ms, total: 203 ms
Wall time: 3.47 s


Again, repeat the same calculations using the imported functions from the "standard" Libra's modules:

In [11]:
%%time 

params_ = { "labels":["F", "C", "H", "O"], "nstates":3,
            "dftb_exe":"/mnt/c/cygwin/home/Alexey-user/Soft/dftb/dftbplus-19.1.x86_64-linux/bin/dftb+",
            "xyz2gen_exe":"/mnt/c/cygwin/home/Alexey-user/Soft/dftb/dftbplus-19.1.x86_64-linux/bin/xyz2gen"
          }

# This is the input in Angstrom
q = MATRIX(12,1)
q.set(0,0, -0.04746);  q.set(1,0,  0.65920);  q.set(2,0,  1.04080)
q.set(3,0, -0.51816);  q.set(4,0,  0.81540);  q.set(5,0,  0.06880)
q.set(6,0,  0.15264);  q.set(7,0,  1.00050);  q.set(8,0,  -0.77170)
q.set(9,0, -1.71206);  q.set(10,0, 0.78520);  q.set(11,0, -0.06860)
q.show_matrix()

# The input is expected to be in Bohr, so lets convert:
q *= units.Angst

id = Py2Cpp_int([0, 0])
obj_dftb = DFTB_methods.run_dftb_adi(q, params_, id )

NameError: name 'comn' is not defined

Analogously to the Psi4 example, the generated object can be used to save the computed results

In [12]:
prefix = "hfco"
obj_dftb.ham_adi.show_matrix(F"{prefix}-ham_adi.txt")
obj_dftb.d1ham_adi[0].show_matrix(F"{prefix}-d1ham_adi-0.txt")

<a name="pes"></a>
[Back to TOC](#TOC)

## 2. PES scan

Now, we will demonstrate the use of the above Hamiltonian calculations functions in building PES for ground excited states of some atomistic systems. 

We will be using the `plot_pes_properties` function of the `libra_py.dynamics_plotting` module. This most basic features of this function has been extensively discueed in [this tutorial](../1_pes_plotting/tutorial.ipynb). Here, we'll highlight a few additional features that can come handy with this function:


<a name="pes_scan-1"></a>
[Back to TOC](#TOC)

### 2.1. EOM-CCSD/STO-3G surfaces of LiH

In this example, we demonstrate the use of the following two parameters to perform a scan w.r.t. one of the DOFs in a many-DOF system like ours:

* `pes_params["active_dof"] = 5` - we are going to vary the DOF with index 5, which corresponds to the z coordinate of the second (H) atom.

* `pes_params["reference_coord"] = q` - we freeze all other DOFs to be what they are in the reference geometry `q` provided here.

<a name="plot_pes_properties-ex1"></a>

In [ ]:
labels = ["Li", "H"]

# This geometry is assumed to be given in Bohrs already
q = MATRIX(6,1)
q.set(0,0, 0.0);  q.set(1,0, 0.0); q.set(2,0, 0.0);
q.set(3,0, 0.0);  q.set(4,0, 0.0); q.set(5,0, 1.5);
q.show_matrix()


model_params = { "labels":labels, "nstates":2, "verbosity":1 }

pes_params = { "ndia":2, "nadi":2, "ndof":6, "active_dof":5,
               "xmin":0.5, "xmax":5.0, "dx":0.5,
               "reference_coord":q,               
               "rep_tdse":1, "rep_ham":1  }

plot_params = { "which_ham_adi":[ [0,0], [1,1] ],
                "which_d1ham_adi":[ [5, 0,0], [5, 1,1] ],
                "colors": colors, "clrs_index": clrs_index
              }


LiH_scan = dynamics_plotting.plot_pes_properties(run_psi4_adi, model_params, pes_params, plot_params )

Doing state 0 energy
Doing state 0 gradient
Doing state 1 energy
Doing state 1 gradient
Doing state 0 energy
Doing state 0 gradient
Doing state 1 energy
Doing state 1 gradient
Doing state 0 energy
Doing state 0 gradient
Doing state 1 energy
Doing state 1 gradient
Doing state 0 energy
Doing state 0 gradient
Doing state 1 energy
Doing state 1 gradient
Doing state 0 energy
Doing state 0 gradient
Doing state 1 energy
Doing state 1 gradient
Doing state 0 energy
Doing state 0 gradient
Doing state 1 energy
Doing state 1 gradient
Doing state 0 energy
Doing state 0 gradient
Doing state 1 energy
Doing state 1 gradient
Doing state 0 energy
Doing state 0 gradient
Doing state 1 energy
Doing state 1 gradient
Doing state 0 energy
Doing state 0 gradient
Doing state 1 energy
Doing state 1 gradient
Doing state 0 energy
Doing state 0 gradient
Doing state 1 energy
Doing state 1 gradient


The generated variable `LiH_scan` contains the geometries of all configurations. 

We can use convert them to a xyz "trajectory" format for further visualization. 

<a name="make_path_xyz2"></a>

To do this, we use the `make_path_xyz2` function of the `libra_py.scan` module

Since the coordinates that we have generated in the `plot_pes_properties` are in Bohrs, and the visualizer of xyz files works best in Angstrom units, we need to use the conversion via `units.Angst` constant.

In [ ]:
labels = ["Li", "H"]
itraj = 0
unit_conversion_factor = 1.0/units.Angst

LiH_xyz = scan.make_path_xyz2(labels, LiH_scan, itraj, unit_conversion_factor)

And now we can visuzlie this scan "trajectory" with py3Dmol:

<a name="md_trajectory-vis"></a>

In [ ]:
view = py3Dmol.view(width=800,height=400)  
view.setBackgroundColor('0xeeeeee')                                     
view.zoomTo()                                                           
view.addModelsAsFrames(LiH_xyz, 'xyz')
view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
view.animate({'reps':0, 'loop':'forward', 'step':1})
view.show()  


<a name="pes_scan-2"></a>
[Back to TOC](#TOC)

### 2.2. TD-DFTB+ surfaces of HFCO

In this example, we are going to use the `run_dftpb_adi` function to construct ground and excited state surfaces of the HFCO molecule. Now, for such a scan it makes sense to choose some chemically-intuitive coordinate, such as bond-length for some of the bonds, say C-O. But how do we do this?

For this purpose, the function `plot_pes_properties` has yet another parameter, that we didn't need in previous situaitons. Namely, the `coord_mapping`. The value that corresponds to this key parameter is actually a Python **function!**. This function is meant to perform the mapping of the 1D scan coordinate on a multi-dimensional coordinate of our system that we are currently interested in. Defining such a function gives you the ultimate control of the scan "reaction" coordinate - it can be as complex as you want. But, again, we are looking to make the C-O bond of the HFCO molecule to be out reaction coordinate.

When the `coord_mapping` parameter is set to something other than `None` (either explicitly or by default), the parameters `reference_coord` and `active_dof` do not matter any more.

For this example, the coordinate mapping function is defined as following:

In [ ]:
def mapping_function(coord):
    """
    Args:
        coord ( int ): the 1D "reaction" coordinate value          
            In this example, "coord" is the degree of the C - O bond elongation
            (1.0 - corresponding to the reference bond value)
            
    Returns:
        MATRIX(ndof, 1): the atomic coordinates of the system        
        
    """
    
    # Reference geometry in Angstrom
    q = MATRIX(12,1)
    q.set(0,0, -0.04746);  q.set(1,0,  0.65920);  q.set(2,0,  1.04080)   # F
    q.set(3,0, -0.51816);  q.set(4,0,  0.81540);  q.set(5,0,  0.06880)   # C
    q.set(6,0,  0.15264);  q.set(7,0,  1.00050);  q.set(8,0,  -0.77170)  # H
    q.set(9,0, -1.71206);  q.set(10,0, 0.78520);  q.set(11,0, -0.06860)  # O
    

    # Out result
    q_res = MATRIX(q)
    
    # r(O) - r(C)
    u = VECTOR()
    u.x = q.get(9,0) - q.get(3,0)
    u.y = q.get(10,0) - q.get(4,0)
    u.z = q.get(11,0) - q.get(5,0)
        
    q_res.set(9, 0,  q.get(3,0) + coord * u.x )
    q_res.set(10, 0,  q.get(4,0) + coord * u.y )
    q_res.set(11, 0,  q.get(5,0) + coord * u.z )
    
    
    # The input is expected to be in Bohr, so lets convert:
    q *= units.Angst
    
    return q_res
    
    

Now, we are ready to use the mapping function in the actual scan calculations:

<a name="plot_pes_properties-ex2"></a>

In [ ]:
labels = ["F", "C", "H", "O"]

q = MATRIX(12,1)
q.set(0,0, -0.04746);  q.set(1,0,  0.65920);  q.set(2,0,  1.04080)
q.set(3,0, -0.51816);  q.set(4,0,  0.81540);  q.set(5,0,  0.06880)
q.set(6,0,  0.15264);  q.set(7,0,  1.00050);  q.set(8,0,  -0.77170)
q.set(9,0, -1.71206);  q.set(10,0, 0.78520);  q.set(11,0, -0.06860)
q *= units.Angst
q.show_matrix()

model_params = { "labels":labels, "nstates":5,
            "dftb_exe":"/mnt/c/cygwin/home/Alexey-user/Soft/dftb/dftbplus-19.1.x86_64-linux/bin/dftb+",
            "xyz2gen_exe":"/mnt/c/cygwin/home/Alexey-user/Soft/dftb/dftbplus-19.1.x86_64-linux/bin/xyz2gen"
          }

pes_params = { "ndia":5, "nadi":5, "ndof":12, "active_dof":5,
               "xmin":0.5, "xmax":5.0, "dx":0.5,               
               "coord_mapping":mapping_function,
               "rep_tdse":1, "rep_ham":1  }

plot_params = { "which_ham_adi":[ [0,0], [1,1], [2,2], [3,3], [4,4] ],
                "which_d1ham_adi":[ [5, 0,0], [5, 1,1], [5, 2, 2], [5, 3, 3], [5, 4, 4] ],
                "colors": colors, "clrs_index": clrs_index
              }


HFCO_scan = dynamics_plotting.plot_pes_properties(run_dftb_adi, model_params, pes_params, plot_params )

As before, we can collect all the scanned geometries and create the xyz-formatter trajectory "file"

In [ ]:
labels = ["F", "C", "H", "O"]
itraj = 0
unit_conversion_factor = 1.0/units.Angst

HFCO_xyz = scan.make_path_xyz2(labels, HFCO_scan, itraj, unit_conversion_factor)

that can be used for visualization of the scan

In [ ]:
view = py3Dmol.view(width=800,height=400)  
view.setBackgroundColor('0xeeeeee')                                     
view.zoomTo()                                                           
view.addModelsAsFrames(HFCO_xyz, 'xyz')
view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
view.animate({'reps':0, 'loop':'forward', 'step':1})
view.show()  